# Домашнее задание №3
Асланов А.Б., ИУ9-11М

- Взять стишок "Дом, который построил Джек";
- Убрать знаки препинания, дефис заменить пробелом;
- Провести:      
  **Аддитивное сглаживание (+1) униграммы;**      
  **Биграммы: сглаживание Уиттена-Белла**;

In [1]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import pandas as pd
import warnings
warnings.filterwarnings("ignore") 

In [2]:
train = '''
Вот дом,
Который построил Джек.
А это пшеница,
Которая в тёмном чулане хранится В доме,
Который построил Джек.
А это весёлая птица-синица, 
Которая часто ворует пшеницу, 
Которая в тёмном чулане хранится В доме,
Который построил Джек. 
'''

In [3]:
test = '''
Вот кот,
Который пугает и ловит синицу, Которая часто ворует пшеницу, Которая в тёмном чулане хранится В доме,
Который построил Джек.
'''

In [4]:
import re

# Заменяем дефис на пробел
train = re.sub('-', ' ', train)
test = re.sub('-', ' ', test)

# Удаляем знаки препинания
train_words = re.split("[ \n;,'`.?!:']", train)
test_words = re.split("[ \n;,'`.?!:']", test)

# Удаляем пустые строки
train_words = list(filter(None, train_words)) #None - потому что фильтрует пустые строки
test_words = list(filter(None, test_words))
train_words = [word.lower() for word in train_words] # все слова приводим к нижнему регистру
test_words = [word.lower() for word in test_words] # все слова приводим к нижнему регистру

In [5]:
print('Обучающая выборка: ', train_words)
print('\n\n')
print('Тестовая выборка: ', test_words)

print('\n')
print('БЕЗ УЧЁТА РЕГИСТРА, С ПОВТОРЯЮЩИМИСЯ СЛОВАМИ')
print('Длина обучающей выборки:', len(train_words), 'слов')
print('Длина тестовой выборки:', len(test_words), 'слов')

Обучающая выборка:  ['вот', 'дом', 'который', 'построил', 'джек', 'а', 'это', 'пшеница', 'которая', 'в', 'тёмном', 'чулане', 'хранится', 'в', 'доме', 'который', 'построил', 'джек', 'а', 'это', 'весёлая', 'птица', 'синица', 'которая', 'часто', 'ворует', 'пшеницу', 'которая', 'в', 'тёмном', 'чулане', 'хранится', 'в', 'доме', 'который', 'построил', 'джек']



Тестовая выборка:  ['вот', 'кот', 'который', 'пугает', 'и', 'ловит', 'синицу', 'которая', 'часто', 'ворует', 'пшеницу', 'которая', 'в', 'тёмном', 'чулане', 'хранится', 'в', 'доме', 'который', 'построил', 'джек']


БЕЗ УЧЁТА РЕГИСТРА, С ПОВТОРЯЮЩИМИСЯ СЛОВАМИ
Длина обучающей выборки: 37 слов
Длина тестовой выборки: 21 слов


## 1. Аддитивное сглажвание униграмм

Исходная вероятность униграммы имеет вид:
\begin{equation}
 P(w_i) = \frac{c_i}{N}
\end{equation}

Новая вероятность униграммы при применении аддитивного сглаживания имеет вид:      
\begin{equation}
P_{LP}(w_i) = \frac{c_i+1}{N+V}
\end{equation}

Здесь $w$ - поданное на вход слово, $c_i$ - частота $i$-го слова, $N$ - общее число слов, $V$ - количество разных слов

Смысл аддитивного сглаживания: к частотам появления всех терминов из словаря искусственно добавляется единица, чтобы новым словам, не встречавшимся ранее в коллекции, приписывалась ненулевая вероятность появления.

**Вероятность униграмм считается по обучающей выборке**, а тестовая необходима для оценки меры качества - перплексии

### 1.1 Исходные вероятности униграмм

In [64]:
print('1. ИСХОДНЫЕ ВЕРОЯТНОСТИ УНИГРАММ')
c = Counter(train_words) # подсчёт числа конкретных слов, встретившихся в тексте
N = len(train_words) # общее число слов

# Вероятности униграмм
Ps = []
for value in c.values():
    Ps.append(value / N)

train_words_no_repeats = [word for word in list(c.keys())]
print('ОБЩЕЕ ЧИСЛО СЛОВ БЕЗ ПОВТОРЕНИЙ В ОБУЧАЮЩЕЙ ВЫБОРКЕ: ', len(train_words_no_repeats))
d = {'Words':train_words_no_repeats, 'Probabilities':Ps}
df = pd.DataFrame(data=d, index=None)
df

1. ИСХОДНЫЕ ВЕРОЯТНОСТИ УНИГРАММ
ОБЩЕЕ ЧИСЛО СЛОВ БЕЗ ПОВТОРЕНИЙ В ОБУЧАЮЩЕЙ ВЫБОРКЕ:  20


,Words,Probabilities
0,вот,0.027027
1,дом,0.027027
2,который,0.081081
3,построил,0.081081
4,джек,0.081081
5,а,0.054054
6,это,0.054054
7,пшеница,0.027027
8,которая,0.081081
9,в,0.108108


### 1.2 Вероятности униграмм при аддитивном сглаживании

In [34]:
print('2. ВЕРОЯТНОСТИ УНИГРАММ ПРИ АДДИТИВНОМ СГЛАЖИВАНИИ (без повторений)')


c_train = Counter(train_words)
c_test = Counter(test_words)
N = len(train_words) # общее число слов
V = len(list(set(train_words))) # число уникальных слов


words_laplace, Ps_laplace = [], []
for word, freq in zip(c_test.keys(), c_test.values()):
    words_laplace.append(word)
    # Если слово из тестовой выборки есть в обучающей выборке, то верояность равна (c_i+1)/(N+V)
    if word in train_words:
        Ps_laplace.append((freq + 1) / (N + V))
    # А если слово не встречалось в обучающей выборке, то присваиваем ему ненулевую вероятность 1/(N+V)
    else:
        Ps_laplace.append(1 / (N + V))



df = pd.DataFrame(data={'Words':words_laplace, 'Probabilities':Ps_laplace}, index=None)
df

2. ВЕРОЯТНОСТИ УНИГРАММ ПРИ АДДИТИВНОМ СГЛАЖИВАНИИ (без повторений)


,Words,Probabilities
0,вот,0.035088
1,кот,0.017544
2,который,0.052632
3,пугает,0.017544
4,и,0.017544
5,ловит,0.017544
6,синицу,0.017544
7,которая,0.052632
8,часто,0.035088
9,ворует,0.035088


### 1.3 Перплексия для униграмм при аддитивном сглаживании
Перплексия - мера качества языковых моделей, показывающая, сколько в среднем мы ожидаем слов после каждого слова в документе.     
Чем меньше перплексия, тем лучше.

In [29]:
import numpy as np


perplexity_laplace = (1/np.prod(df['Probabilities'])) ** (1/len(test_words))
print('Перплексия для униграмм (после сглаживания Лапласа): ', perplexity_laplace)

Перплексия для униграмм (после сглаживания Лапласа):  19.657477821544756


## 2. Сглаживание Уиттена-Белла

Сглаживание Уиттена-Белла нужно для включения в модель фактора разнообразия возможных предсказываемых слов.    
Подсчитаем количество биграмм и униграмм:

In [55]:
# Построим биграммы для обучающей и тестовой выборок
train_text = ''
for word in train_words:
    train_text += str(word.lower())
    train_text += ' '

    
test_text = ''
for word in test_words:
    test_text += str(word.lower())
    test_text += ' '


train_tokens = nltk.word_tokenize(train_text)
train_bigrams = list(nltk.bigrams(train_tokens))
train_unigrams = list(ngrams(train_tokens, 1))

test_tokens = nltk.word_tokenize(test_text)
test_bigrams = list(nltk.bigrams(test_tokens))
test_unigrams = list(ngrams(test_tokens, 1))

In [51]:
bigrams = Counter(train_bigrams)
unigrams = Counter(train_unigrams)


Ps_Witten = []
for bigram in bigrams:
    for unigram in unigrams:
        if bigram[0] == unigram[0]:
            Ps_Witten.append(bigrams[bigram] / (bigrams[bigram] + unigrams[unigram]))


d = {'Words':list(bigrams.keys()), 'Probabilities (Witten-Bell)':Ps_Witten}
df = pd.DataFrame(data=d, index=None)
print('3. ВЕРОЯТНОСТИ БИГРАММ ПРИ СГЛАЖИВАНИИ УИТТЕНА-БЕЛЛА')
df

3. ВЕРОЯТНОСТИ БИГРАММ ПРИ СГЛАЖИВАНИИ УИТТЕНА-БЕЛЛА


,Words,Probabilities (Witten-Bell)
0,"(вот, дом)",0.500000
1,"(дом, который)",0.500000
2,"(который, построил)",0.500000
3,"(построил, джек)",0.500000
4,"(джек, а)",0.400000
5,"(а, это)",0.500000
6,"(это, пшеница)",0.333333
7,"(пшеница, которая)",0.500000
8,"(которая, в)",0.400000
9,"(в, тёмном)",0.333333


In [56]:
perplexity_witten = (1/np.prod(Ps_Witten)) ** (1/len(test_bigrams))
print('Перплексия для биграмм (после сглаживания Виттена-Белла): ', perplexity_witten)

Перплексия для биграмм (после сглаживания Виттена-Белла):  2.547682171386066


### 2.3 Вероятность исходных биграмм

Вероятность исходных биграмм имеет вид:
\begin{equation}
P(w_n|w_{n-1}) = \frac{c(w_{n}w_{n-1})}{c(w_{n-1})}
\end{equation}

In [57]:
bigrams = Counter(train_bigrams)
unigrams = Counter(train_unigrams)

Ps = []
for bigram in bigrams:
    for unigram in unigrams:
        if bigram[0] == unigram[0]: # если первое слово биграммы совпадает с униграммой...
            Ps.append(bigrams[bigram] / unigrams[unigram]) # считаем вероятность как отношение частоты встречаемости
                                                           # биграммы к частоте встречаемости униграммы


d = {'Words':list(bigrams.keys()), 'Probabilities':Ps}
df = pd.DataFrame(data=d, index=None)
df

,Words,Probabilities
0,"(вот, дом)",1.000000
1,"(дом, который)",1.000000
2,"(который, построил)",1.000000
3,"(построил, джек)",1.000000
4,"(джек, а)",0.666667
5,"(а, это)",1.000000
6,"(это, пшеница)",0.500000
7,"(пшеница, которая)",1.000000
8,"(которая, в)",0.666667
9,"(в, тёмном)",0.500000


### 2.4 Вероятности биграмм при сглаживании по Лапласу
Вероятность биграмм при сглаживании по Лапласу имеет вид:
\begin{equation}
P(w_n|w_{n-1}) = \frac{c(w_{n}w_{n-1})+ 1}{c(w_{n-1}) + V^2}
\end{equation}

Здесь $V$ - количество разных слов

In [80]:
print('2. ВЕРОЯТНОСТИ УНИГРАММ ПРИ АДДИТИВНОМ СГЛАЖИВАНИИ (без повторений)')


c_train = Counter(train_words)
c_test = Counter(test_words)
V = len(list(set(train_words))) # число уникальных слов


bigrams_train = Counter(train_bigrams)
unigrams_train = Counter(train_unigrams)
bigrams_test = Counter(test_bigrams)
unigrams_test = Counter(test_unigrams)

Ps_laplace_bigrams = []
bigrams_test_laplace = []
for bigram_test, freq in zip(bigrams_test.keys(), bigrams_test.values()):
    for unigram_test in unigrams_test.keys():
        if bigram_test[0] == unigram_test[0]: # если первое слово биграммы совпадает с униграммой...
            bigrams_test_laplace.append(bigram_test)
            
            # Если биграмма из тестовой выборки есть в обучающей выборке, то верояность равна (c_{i|i-1}+1)/(c_{i-1}+V^2)
            if bigram_test in bigrams_train:
                Ps_laplace_bigrams.append((bigrams_test[bigram_test] + 1) / (unigrams_test[unigram_test] + V**2))
                
            # Если биграммы из тестовой выборки нет в обучающей выборке, то присваивается ненулевая вероятность 1/V^2
            else:
                Ps_laplace_bigrams.append(1 / V**2)
    

df = pd.DataFrame(data={'Bigrams':bigrams_test_laplace, 'Probabilities':Ps_laplace_bigrams}, index=None)
df

2. ВЕРОЯТНОСТИ УНИГРАММ ПРИ АДДИТИВНОМ СГЛАЖИВАНИИ (без повторений)


,Bigrams,Probabilities
0,"(вот, кот)",0.002500
1,"(кот, который)",0.002500
2,"(который, пугает)",0.002500
3,"(пугает, и)",0.002500
4,"(и, ловит)",0.002500
5,"(ловит, синицу)",0.002500
6,"(синицу, которая)",0.002500
7,"(которая, часто)",0.004975
8,"(часто, ворует)",0.004988
9,"(ворует, пшеницу)",0.004988


### Выводы

1. Сглаживание - повышение вероятности некоторых $n$-грамм засчет понижения вероятности других.     
2. Смысл аддитивного сглаживания: к частотам появления всех терминов из словаря искусственно добавляется единица, чтобы новым словам, не встречавшимся ранее в коллекции, приписывалась ненулевая вероятность появления.     
3. Сглаживание Уиттена-Белла нужно для включения в модель фактора разнообразия возможных предсказываемых слов.      
4. Перплексия - мера качества - показывает, сколько в среднем мы ожидаем слов после каждого слова в документе. Чем она меньше, тем лучше.

#### Результаты
Перплексия для униграмм (после сглаживания Лапласа) $\approx$ **19.657**;    
Перплексия для биграмм (после сглаживания Виттена-Белла) $\approx$  **2.547**